# Packages

In [40]:
import geopandas as gpd
import os


In [41]:

# Define file path
file_path = r"G:\Shared drives\Wellcome Trust Project Data\0_source_data\GiGL land use data\GiGL_OpenSpace_Sites_All_region\GiGL_OpenSpace_Sites_All_region.shp"

# Read the spatial data
d = gpd.read_file(file_path)


# LC scenarios 

### Define and generate opportunity land polygons 

In [42]:

# print(sorted(d["POSGrade"].dropna().unique()))

# # Define categories to filter
# PrimaryUse_select = [
#     "Disused quarry/gravel pit", "Disused railway trackbed", "Land reclamation",
#     "Other hard surfaced areas", "Other recreational", "Road island/verge",
#     "Vacant land"
# ]

# # Filter data based on the selected categories
# d_opportunityLC = d[d["PrimaryUse"].isin(PrimaryUse_select)]

# # # Display the filtered data
# # print(d_opportunityLC.head())



# ## Save filtered shp

# output_path = "G:/Shared drives/Wellcome Trust Project Data/1_preprocess/UrbanCoolingModel/GiGL_OpenSpace_Sites_opportunityLC.shp"

# # Save filtered data as a new shapefile
# d_opportunityLC.to_file(output_path, driver="ESRI Shapefile")

# print(f"Filtered shapefile saved at: {output_path}")


## Load LC data

In [43]:
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.mask import mask
from rasterio.features import rasterize

# --- File Paths ---
landcover_raster_path     = r"G:\Shared drives\Wellcome Trust Project Data\1_preprocess\UrbanCoolingModel\ESA_WorldCover_10m_2021_v200_Mosaic_Mask_proj.tif"
shp_opportunityLC_path    = r"G:\Shared drives\Wellcome Trust Project Data\1_preprocess\UrbanCoolingModel\GiGL_OpenSpace_Sites_opportunityLC.shp"
lc_scenario_opp2treecover = r"G:\Shared drives\Wellcome Trust Project Data\1_preprocess\UrbanCoolingModel\ESA_WorldCover_10m_2021_update_scenario_opp2treecover.tif"

# --- Step 1: Load the Raster ---
with rasterio.open(landcover_raster_path) as src:
    landcover_data = src.read(1)      # Read the first band
    landcover_meta = src.meta.copy()  # Copy metadata
    landcover_crs = src.crs           # Get raster CRS
    transform = src.transform         # pixel size, georeferencing
    nodata = src.nodata


# --- Step 2: Load the Shapefile ---
d_opportunityLC = gpd.read_file(shp_opportunityLC_path)

## total area calculation
# Ensure CRS is projected (replace EPSG:XXXX with a suitable projection for your area, e.g., EPSG:5070 for US)
if d_opportunityLC.crs.is_geographic:
    d_opportunityLC = d_opportunityLC.to_crs(epsg=5070)  # Albers Equal Area for US

# Calculate total area in square meters
total_area_m2 = d_opportunityLC.geometry.area.sum()

# Optionally convert to square kilometers or hectares
total_area_km2 = total_area_m2 / 1e6
total_area_ha = total_area_m2 / 10000

print(f"Total area of opportunity land cover: {total_area_km2:,.2f} km²")
print(f"Total area of opportunity land cover: {total_area_ha:,.2f} hectares")

Total area of opportunity land cover: 33.40 km²
Total area of opportunity land cover: 3,339.98 hectares


## % of each LC - baseline

Using ESA LC data

In [44]:
from collections import Counter

# Define labels
land_cover_labels = {
    1: 'Tree cover',
    2: 'Shrubland',
    3: 'Grassland',
    4: 'Cropland',
    5: 'Built-up',
    6: "Bare / sparse vegetation",
    7: "Snow and ice",
    8: "Permanent water bodies",
    9: "Herbaceous wetland",
    10: "Moss and lichen"
}


# Flatten the array and remove NoData values
valid_pixels = landcover_data[landcover_data != nodata].flatten()

# Count frequency of each land use type
counts = Counter(valid_pixels)

# Total number of valid pixels
total_pixels = sum(counts.values())

# Calculate proportions
proportions = {land_use: count / total_pixels for land_use, count in counts.items()}

# Print result
# Print with labels
print("Land use proportions:")
for lu_type, prop in proportions.items():
    label = land_cover_labels.get(lu_type, f"Unknown ({lu_type})")
    print(f"{label}: {prop:.2%}")

Land use proportions:
Built-up: 43.26%
Tree cover: 33.89%
Grassland: 18.15%
Cropland: 2.51%
Bare / sparse vegetation: 0.07%
Permanent water bodies: 2.09%
Herbaceous wetland: 0.04%
Shrubland: 0.00%


## Create LC scenario 1 - to pavement 

Turn tree cover and shrub to built-up land

In [45]:
# --- Step 2: Remap classes (1,2) -> 5 while preserving NoData ---
remapped = landcover_data.copy()

# mask to protect NoData
if nodata is not None:
    valid_mask = landcover_data != nodata
else:
    valid_mask = np.ones_like(landcover_data, dtype=bool)

# change Tree cover (1) and Shrubland (2) to Built-up (5)
to_built = (landcover_data == 1) | (landcover_data == 2)
remapped = np.where(valid_mask & to_built, 5, landcover_data)

# --- Step 3: Write out the scenario raster ---
scenario_path = landcover_raster_path.replace(".tif", "_BuiltUpScenario.tif")

# keep original dtype; add compression if you like
landcover_meta.update(
    dtype=remapped.dtype,
    nodata=nodata,
    compress="lzw"
)

with rasterio.open(scenario_path, "w", **landcover_meta) as dst:
    dst.write(remapped, 1)

print(f"Scenario saved to: {scenario_path}")



# --- Optional: quick summary of the change and area ---
changed_pixels = int((valid_mask & to_built).sum())
px_area = abs(transform.a) * abs(transform.e)  # m² per pixel (if in a projected CRS)
changed_area_km2 = changed_pixels * px_area / 1e6 if landcover_crs.is_projected else None

print(f"Pixels converted to Built-up: {changed_pixels:,}")
if changed_area_km2 is not None:
    print(f"Estimated area converted: {changed_area_km2:,.2f} km² "
          f"(assuming projected CRS: {landcover_crs})")
else:
    print("Area estimate skipped (CRS is geographic; reproject raster to a projected CRS for area).")

# --- Optional: update labels for the new scenario (if you keep a legend) ---
land_cover_labels_scenario = {k: v for k, v in land_cover_labels.items()}
land_cover_labels_scenario[1] = "Built-up"
land_cover_labels_scenario[2] = "Built-up"
# Note: The raster now has only class 5 where 1/2 used to be; this relabel helps for plotting legends.



Scenario saved to: G:\Shared drives\Wellcome Trust Project Data\1_preprocess\UrbanCoolingModel\ESA_WorldCover_10m_2021_v200_Mosaic_Mask_proj_BuiltUpScenario.tif
Pixels converted to Built-up: 12,399,927
Estimated area converted: 540.59 km² (assuming projected CRS: EPSG:27700)


### % of each LC - function 

In [46]:
import numpy as np
import pandas as pd

def summarize_classes(arr, labels, nodata=None, px_area_m2=None, sort_by="class"):
    """
    Summarize class counts, proportions, and (optional) areas for a labeled raster.
    - arr: 2D numpy array of class codes
    - labels: dict {code: "label"}
    - nodata: value to exclude (or None)
    - px_area_m2: pixel area in m² (optional; requires projected CRS)
    - sort_by: "class" or "proportion"
    """
    if nodata is not None:
        mask = arr != nodata
    else:
        mask = np.ones_like(arr, dtype=bool)

    vals, counts = np.unique(arr[mask], return_counts=True)
    total = counts.sum()

    # Build DataFrame
    df = pd.DataFrame({
        "class_code": vals,
        "label": [labels.get(int(v), f"Class {int(v)}") for v in vals],
        "count": counts,
        "proportion": counts / total
    })
    df["percent"] = 100 * df["proportion"]

    if px_area_m2 is not None:
        df["area_m2"] = df["count"] * px_area_m2
        df["area_ha"] = df["area_m2"] / 10_000
        df["area_km2"] = df["area_m2"] / 1e6

    if sort_by == "proportion":
        df = df.sort_values("proportion", ascending=False)
    else:
        df = df.sort_values("class_code")

    # Nice rounding for display
    df["proportion"] = df["proportion"].round(6)
    df["percent"] = df["percent"].round(3)
    for col in ("area_m2", "area_ha", "area_km2"):
        if col in df.columns:
            df[col] = df[col].round(2)

    return df.reset_index(drop=True)




### % of each LC - run 

In [47]:


# ---- Compute pixel area if projected (optional) ----
px_area_m2 = None
if landcover_crs and landcover_crs.is_projected:
    # rasterio Affine: transform.a = pixel width, transform.e = pixel height (negative)
    px_area_m2 = abs(transform.a) * abs(transform.e)

# ---- Summaries: BEFORE (original) and AFTER (remapped) ----
summary_before = summarize_classes(
    landcover_data,
    land_cover_labels,
    nodata=nodata,
    px_area_m2=px_area_m2,
    sort_by="class"
)

# Optional: if you defined land_cover_labels_scenario; otherwise reuse land_cover_labels
labels_after = globals().get("land_cover_labels_scenario", land_cover_labels)

summary_after = summarize_classes(
    remapped,
    labels_after,
    nodata=nodata,
    px_area_m2=px_area_m2,
    sort_by="class"
)

print("\n=== Class proportions BEFORE scenario ===")
print(summary_before.to_string(index=False))

print("\n=== Class proportions AFTER scenario ===")
print(summary_after.to_string(index=False))

# ---- Optional: quick change report for Tree->Built-up and Shrubland->Built-up ----
def class_count(df, code):
    row = df.loc[df["class_code"] == code, "count"]
    return int(row.iloc[0]) if len(row) else 0

c_tree_before = class_count(summary_before, 1)
c_shrub_before = class_count(summary_before, 2)
c_built_before = class_count(summary_before, 5)

c_tree_after  = class_count(summary_after, 1)
c_shrub_after = class_count(summary_after, 2)
c_built_after = class_count(summary_after, 5)

print("\n=== Change summary ===")
print(f"Tree cover (1): {c_tree_before:,} -> {c_tree_after:,}")
print(f"Shrubland  (2): {c_shrub_before:,} -> {c_shrub_after:,}")
print(f"Built-up   (5): {c_built_before:,} -> {c_built_after:,} "
      f"(+{(c_built_after - c_built_before):,})")

# ---- Optional: save to CSV ----
out_csv_before = scenario_path.replace("_BuiltUpScenario.tif", "_class_summary_BEFORE.csv")
out_csv_after  = scenario_path.replace("_BuiltUpScenario.tif", "_class_summary_AFTER.csv")
summary_before.to_csv(out_csv_before, index=False)
summary_after.to_csv(out_csv_after, index=False)
print(f"\nSaved summaries:\n- {out_csv_before}\n- {out_csv_after}")



=== Class proportions BEFORE scenario ===
 class_code                    label    count  proportion  percent      area_m2  area_ha  area_km2
          1               Tree cover 12399905    0.338917   33.892 540590568.05 54059.06    540.59
          2                Shrubland       22    0.000001    0.000       959.12     0.10      0.00
          3                Grassland  6638951    0.181458   18.146 289434015.21 28943.40    289.43
          4                 Cropland   918903    0.025116    2.512  40060814.56  4006.08     40.06
          5                 Built-up 15825867    0.432557   43.256 689949998.12 68995.00    689.95
          6 Bare / sparse vegetation    26810    0.000733    0.073   1168818.08   116.88      1.17
          8   Permanent water bodies   763389    0.020865    2.087  33280972.16  3328.10     33.28
          9       Herbaceous wetland    12956    0.000354    0.035    564834.28    56.48      0.56

=== Class proportions AFTER scenario ===
 class_code             

## Create LC scenario 2 - more tree cover

In [48]:

# --- Step 3: Reproject Shapefile if Needed ---
# (Skip if either CRS is None; otherwise align to raster CRS)
if d_opportunityLC.crs and (d_opportunityLC.crs != landcover_crs):
    d_opportunityLC = d_opportunityLC.to_crs(landcover_crs)

# Drop empty/invalid geometries (common source of rasterize errors)
d_opportunityLC = d_opportunityLC[d_opportunityLC.geometry.notna() & ~d_opportunityLC.geometry.is_empty]

# --- Step 4: Rasterize the Shapefile ---
shape_mask = rasterize(
    [(geom, 1) for geom in d_opportunityLC.geometry],
    out_shape=landcover_data.shape,
    transform=landcover_meta["transform"],
    fill=0,
    dtype="uint8"
)


# --- Step 5: Apply the Mask to Update Land Cover Values ---
tree_cover_code = 1  # ESA WorldCover: Tree cover
remapped2 = landcover_data.copy()

# Protect NoData if present
if nodata is not None:
    valid_mask = (landcover_data != nodata)
else:
    valid_mask = np.ones_like(landcover_data, dtype=bool)

target_mask = (shape_mask == 1) & valid_mask
remapped2[target_mask] = tree_cover_code



# --- Step 6: Save the Updated Raster ---
landcover_meta.update(dtype=rasterio.uint8, compress="lzw")  # Ensure correct datatype

meta_out = landcover_meta.copy()
meta_out.update(
    dtype=rasterio.uint8,
    compress="lzw",
    nodata=nodata
)


with rasterio.open(lc_scenario_opp2treecover, "w", **meta_out) as dst:
    dst.write(remapped2.astype(rasterio.uint8), 1)

print(f"Updated land cover raster saved at: {lc_scenario_opp2treecover}")


# Scenario labels
land_cover_labels_scenario2 = land_cover_labels.copy()


Updated land cover raster saved at: G:\Shared drives\Wellcome Trust Project Data\1_preprocess\UrbanCoolingModel\ESA_WorldCover_10m_2021_update_scenario_opp2treecover.tif


### % of LC change - run

In [49]:

# ---- Compute pixel area if projected (optional) ----
px_area_m2 = None
if landcover_crs and landcover_crs.is_projected:
    # rasterio Affine: transform.a = pixel width, transform.e = pixel height (negative)
    px_area_m2 = abs(transform.a) * abs(transform.e)

# ---- Summaries: BEFORE (original) and AFTER (remapped) ----
summary_before = summarize_classes(
    landcover_data,
    land_cover_labels,
    nodata=nodata,
    px_area_m2=px_area_m2,
    sort_by="class"
)

# Optional: if you defined land_cover_labels_scenario; otherwise reuse land_cover_labels
labels_after = globals().get("land_cover_labels_scenario2", land_cover_labels)

summary_after = summarize_classes(
    remapped2,
    labels_after,
    nodata=nodata,
    px_area_m2=px_area_m2,
    sort_by="class"
)

print("\n=== Class proportions BEFORE scenario ===")
print(summary_before.to_string(index=False))

print("\n=== Class proportions AFTER scenario ===")
print(summary_after.to_string(index=False))

# ---- Optional: quick change report for Tree->Built-up and Shrubland->Built-up ----
def class_count(df, code):
    row = df.loc[df["class_code"] == code, "count"]
    return int(row.iloc[0]) if len(row) else 0

c_tree_before = class_count(summary_before, 1)
c_shrub_before = class_count(summary_before, 2)
c_built_before = class_count(summary_before, 5)

c_tree_after  = class_count(summary_after, 1)
c_shrub_after = class_count(summary_after, 2)
c_built_after = class_count(summary_after, 5)

print("\n=== Change summary ===")
print(f"Tree cover (1): {c_tree_before:,} -> {c_tree_after:,}")
print(f"Shrubland  (2): {c_shrub_before:,} -> {c_shrub_after:,}")
print(f"Built-up   (5): {c_built_before:,} -> {c_built_after:,} "
      f"(+{(c_built_after - c_built_before):,})")

# ---- Optional: save to CSV ----
out_csv_before = scenario_path.replace("_BuiltUpScenario.tif", "_class_summary_BEFORE.csv")
out_csv_after  = scenario_path.replace("_BuiltUpScenario.tif", "_class_summary_AFTER.csv")
summary_before.to_csv(out_csv_before, index=False)
summary_after.to_csv(out_csv_after, index=False)
print(f"\nSaved summaries:\n- {out_csv_before}\n- {out_csv_after}")


=== Class proportions BEFORE scenario ===
 class_code                    label    count  proportion  percent      area_m2  area_ha  area_km2
          1               Tree cover 12399905    0.338917   33.892 540590568.05 54059.06    540.59
          2                Shrubland       22    0.000001    0.000       959.12     0.10      0.00
          3                Grassland  6638951    0.181458   18.146 289434015.21 28943.40    289.43
          4                 Cropland   918903    0.025116    2.512  40060814.56  4006.08     40.06
          5                 Built-up 15825867    0.432557   43.256 689949998.12 68995.00    689.95
          6 Bare / sparse vegetation    26810    0.000733    0.073   1168818.08   116.88      1.17
          8   Permanent water bodies   763389    0.020865    2.087  33280972.16  3328.10     33.28
          9       Herbaceous wetland    12956    0.000354    0.035    564834.28    56.48      0.56

=== Class proportions AFTER scenario ===
 class_code             